# *Insight*-HXMT 相位分解谱处理示例
## ——[概览](#概览)、[数据预处理](#数据预处理)、[计时分析](#计时分析)、[能谱分析](#能谱分析)

庹攸隶(tuoyl@ihep.ac.cn)
##### 最终结果：使用慧眼一次 Crab 的观测数据，产生 Crab 脉冲星的相位分解谱

## 概览
### 准备工作
该 Jupyter 文本使用 Python3 环境，若想执行以下所有命令，需要如下操作：
* 安装并初始化 HXMTDAS 环境（例如能在终端中运行```hepical```命令）
* 使用 Python3.* 版本，并安装有 astropy, numpy, matplotlib 模块
    （若使用 conda 环境，请在 environment.yml 文件所在路径执行 ``` conda env create -f environment.yml ``` 安装名为 hxmt_analysis 的环境，然后执行 ```conda activate hxmt_analysis```
    <div class="alert alert-block alert-info">
<b>NOTES
    :</b> 该介绍中涉及到的命令，你可以在 Jupyter 中使用 Shift+Enter 逐条执行。你同时完全可以在终端中执行所有命令。
</div>
    
### 目标
* **数据预处理**：使用慧眼用户处理软件（HXMTDAS v2.01）产生用于分析的数据产品
* **计时分析**：使用 Crab 的星历（ephemeris）产生 Crab 的脉冲轮廓
* **能谱分析**：对轮廓分成多个相位区间，得到各个区间的能谱及背景能谱

## 数据预处理
数据预处理针对三个载荷高能、中能和低等载荷，分别是：
* [高能载荷(HE)数据处理](#高能载荷(HE)数据处理)
* [中能载荷(ME)数据处理](#中能载荷(ME)数据处理)
* [低能载荷(LE)数据处理](#低能载荷(LE)数据处理)

### 高能载荷(HE)数据处理
### hepical

In [4]:
!hepical evtfile=./data/HE/HXMT_P010129900101_HE-Evt_FFFFFF_V1_L1P.FITS outfile=./data/HE/he_pi.fits clobber=yes

hepical : ##############################################
hepical : HXMT HE task, hepical is running
hepical : PILParSet Warning: parameter 'clobber' set to yes!
hepical : PILParSet Warning: the file ./data/HE/he_pi.fits will be overwritten!
hepical : HxmtCaldb Error: There are not files that satisfied the selection criteria!
hepical : HxmtCaldb Error: 'Telescope' HXMT 'instrument' HE 'detname' NONE 'filter' NONE 'codename' CHAN2PI_0' 
hepical : hepical: Error: Unable to get 'gain (codename:CHAN2PI_0)' file named ''!
hepical : HXMT HE task, hepical is running unsuccessfully!
hepical : ##############################################


<div class="alert alert-block alert-warning">
<b>NOTES:</b> 若出现报错 hepical : hepical: Error: Unable to get 'gain (codename:CHAN2PI_0)' file named ''! 则根据 CALDB 中 gainfile 的路径手动指定，执行如下命令。
</div>

In [7]:
!hepical evtfile=./data/HE/HXMT_P010129900101_HE-Evt_FFFFFF_V1_L1P.FITS gainfile=/Users/tuoyouli/Documents/hxmtsoft_newtest/CALDB/data/hxmt/he/bcf/hxmt_he_gain_20171030_v1.fits outfile=./data/HE/he_pi.fits clobber=yes

hepical : ##############################################
hepical : HXMT HE task, hepical is running
[####################################################################################################][100%]
hepical : HXMT HE task, all events from raw evt file is 145630697!
hepical : HXMT HE task, glitch removal events is 636135!
hepical : HXMT HE task, hepical is running successfully!
hepical : ##############################################


运行该命令根据不同的计算机性能，通常会占用你2-3分钟的时间。输出产生一个新的事例文件，命名为 he_pi.fits。

In [8]:
!ls -trl ./data/HE/

total 9658648
-rwx------  1 tuoyouli  staff     1143360 Aug 19 15:31 HXMT_P010129900101_HE-HV_FFFFFF_V1_L1P.FITS
-rwx------  1 tuoyouli  staff      469440 Aug 19 15:31 HXMT_P010129900101_HE-DTime_FFFFFF_V1_L1P.FITS
-rwx------  1 tuoyouli  staff      109440 Aug 19 15:31 HXMT_P010129900101_HE-PM_FFFFFF_V1_L1P.FITS
-rwx------  1 tuoyouli  staff     1673280 Aug 19 15:31 HXMT_P010129900101_HE-TH_FFFFFF_V1_L1P.FITS
-rw-r--r--  1 tuoyouli  staff  2330262720 Aug 19 15:31 HXMT_P010129900101_HE-Evt_FFFFFF_V1_L1P.FITS
-rwx------  1 tuoyouli  staff      149760 Aug 19 15:31 HXMT_P010129900101_HE-InsStat_FFFFFF_V1_L1P.FITS
-rw-r--r--  1 tuoyouli  staff  2610072000 Aug 19 20:18 he_pi.fits


### hegtigen
生成 HE 载荷的好时间文件（GTIs）

In [11]:
!hegtigen hvfile=./data/HE/HXMT_P010129900101_HE-HV_FFFFFF_V1_L1P.FITS tempfile=./data/HE/HXMT_P010129900101_HE-TH_FFFFFF_V1_L1P.FITS ehkfile=./data/AUX/HXMT_P010129900101_HE-EHK_FFFFFF_V1_L1P.FITS outfile=./data/HE/he_gti.fits defaultexpr="NONE" expr="ELV>10&&COR>8&&T_SAA>=300&&TN_SAA>=300&&ANG_DIST<=0.04" clobber=yes

hegtigen : ##############################################
hegtigen : HXMT HE task, hegtigen is running
hegtigen : PILParSet Warning: parameter 'clobber' set to yes!
hegtigen : PILParSet Warning: the file ./data/HE/he_gti.fits will be overwritten!
hegtigen : HXMT HE task, hegtigen is running successfully!
hegtigen : ##############################################


我们推荐的对于涉及到能谱分析的好时间判选条件为```ELV>10&&COR>8&&T_SAA>=300&&TN_SAA>=300&&ANG_DIST<=0.04```。
同样的，你可以查看输出文件，我们这一步产生了一个命名为 he_gti.fits 的 FITS 文件。

In [12]:
!ls -trl ./data/HE/

total 9658688
-rwx------  1 tuoyouli  staff     1143360 Aug 19 15:31 HXMT_P010129900101_HE-HV_FFFFFF_V1_L1P.FITS
-rwx------  1 tuoyouli  staff      469440 Aug 19 15:31 HXMT_P010129900101_HE-DTime_FFFFFF_V1_L1P.FITS
-rwx------  1 tuoyouli  staff      109440 Aug 19 15:31 HXMT_P010129900101_HE-PM_FFFFFF_V1_L1P.FITS
-rwx------  1 tuoyouli  staff     1673280 Aug 19 15:31 HXMT_P010129900101_HE-TH_FFFFFF_V1_L1P.FITS
-rw-r--r--  1 tuoyouli  staff  2330262720 Aug 19 15:31 HXMT_P010129900101_HE-Evt_FFFFFF_V1_L1P.FITS
-rwx------  1 tuoyouli  staff      149760 Aug 19 15:31 HXMT_P010129900101_HE-InsStat_FFFFFF_V1_L1P.FITS
-rw-r--r--  1 tuoyouli  staff  2610072000 Aug 19 20:18 he_pi.fits
-rw-r--r--  1 tuoyouli  staff       17280 Aug 19 20:19 he_gti.fits


### hescreen
根据 GTIs 选择出符合要求的好事例

In [1]:
!hescreen evtfile=./data/HE/he_pi.fits gtifile=./data/HE/he_gti.fits outfile=./data/HE/he_screen.fits userdetid=0-17 minPI=0 maxPI=255 clobber=yes

hescreen : ##############################################
hescreen : HXMT HE task, hescreen is running
hescreen : PILParSet Warning: parameter 'clobber' set to yes!
hescreen : PILParSet Warning: the file ./data/HE/he_screen.fits will be overwritten!
hescreen : hescreen: User detector selection:
hescreen : HxmtBadDetector Info: there is no bad detector!
[####################################################################################################][100%]
hescreen : HEScreen task: 
hescreen : Detector ID '0' has event number :237400
hescreen : Detector ID '1' has event number :244273
hescreen : Detector ID '2' has event number :263201
hescreen : Detector ID '3' has event number :247841
hescreen : Detector ID '4' has event number :236576
hescreen : Detector ID '5' has event number :242721
hescreen : Detector ID '6' has event number :252674
hescreen : Detector ID '7' has event number :237050
hescreen : Detector ID '8' has event number :261260
hescreen : Detector ID '9' has event numb

运行 hescreen 命令大约会占用你1分钟时间，输出文件为 he_screen.fits。我们这里选择了探测器编号0-17，即全选，选择了能道0-255，也是全选。如果你选择产生不同能段的光子，则可以修改 minPI 和 maxPI 的值。

### hespecgen 
生成能谱文件（spectra）

In [1]:
!hespecgen evtfile=./data/HE/he_screen.fits outfile=./data/HE/he_spec deadfile=./data/HE/HXMT_P010129900101_HE-DTime_FFFFFF_V1_L1P.FITS userdetid="0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17" starttime=0 stoptime=0 minPI=0 maxPI=255 clobber=yes

hespecgen : HXMT HE task, hespecgen is running
hespecgen : hespecgen: User detector selection:
[####################################################################################################][100%]
hespecgen : HXMT HE task, hespecgen is running successfully!
hespecgen : ##############################################


我们在这一步生成了 HE 的18个探测器的能谱，通过 userdetid 参数选择探测器。使用分号(;)将各个探测器编号做分割，使我们产生了18个能谱，而不是18个探测器的总能谱。可以查看一下我们产生的结果，是18个以```he_spec```为前缀的```.pha```文件。能谱的结果可以在[能谱分析](#能谱分析)一节查看，这里暂不做展开。

In [6]:
!ls -tr ./data/HE/

HXMT_P010129900101_HE-HV_FFFFFF_V1_L1P.FITS	  he_spec_g4_4.pha
HXMT_P010129900101_HE-DTime_FFFFFF_V1_L1P.FITS	  he_spec_g3_3.pha
he_pi.fits					  he_spec_g2_2.pha
he_gti.fits					  he_spec_g17_17.pha
HXMT_P010129900101_HE-PM_FFFFFF_V1_L1P.FITS	  he_spec_g16_16.pha
he_screen.fits					  he_spec_g15_15.pha
HXMT_P010129900101_HE-TH_FFFFFF_V1_L1P.FITS	  he_spec_g14_14.pha
HXMT_P010129900101_HE-InsStat_FFFFFF_V1_L1P.FITS  he_spec_g13_13.pha
HXMT_P010129900101_HE-Evt_FFFFFF_V1_L1P.FITS	  he_spec_g12_12.pha
he_spec_g9_9.pha				  he_spec_g1_1.pha
he_spec_g8_8.pha				  he_spec_g11_11.pha
he_spec_g7_7.pha				  he_spec_g10_10.pha
he_spec_g6_6.pha				  he_spec_g0_0.pha
he_spec_g5_5.pha


<div class="alert alert-block alert-info">
<b>NOTES:</b> 我们之所以要产生18个能谱而不是一个总谱，是由于各个探测器的响应矩阵不一样，我们在之后会根据各个探测器的能谱产生其各自的响应（盲探测器不产生）。最后会将能谱和响应矩阵合并。
</div>

In [1]:
!ls ./data/HE/he_spec*.pha | sort -V > ./data/HE/he_spec.txt
!cat ./data/HE/he_spec.txt

./data/HE/he_spec_g0_0.pha
./data/HE/he_spec_g1_1.pha
./data/HE/he_spec_g2_2.pha
./data/HE/he_spec_g3_3.pha
./data/HE/he_spec_g4_4.pha
./data/HE/he_spec_g5_5.pha
./data/HE/he_spec_g6_6.pha
./data/HE/he_spec_g7_7.pha
./data/HE/he_spec_g8_8.pha
./data/HE/he_spec_g9_9.pha
./data/HE/he_spec_g10_10.pha
./data/HE/he_spec_g11_11.pha
./data/HE/he_spec_g12_12.pha
./data/HE/he_spec_g13_13.pha
./data/HE/he_spec_g14_14.pha
./data/HE/he_spec_g15_15.pha
./data/HE/he_spec_g16_16.pha
./data/HE/he_spec_g17_17.pha


### herspgen
产生除盲探测器外，所有探测器各自的响应矩阵

In [1]:
phalist = open("./data/HE/he_spec.txt")
for phafile,i in zip(phalist,range(18)):
    if i == 16:continue # 16号盲探测器不产生响应矩阵
    herspgen_text = "herspgen phafile=%s outfile=./data/HE/he_rsp_g%s.fits attfile=%s ra=-1 dec=-91 clobber=yes"%(phafile[0:-1], str(i), "./data/ACS/HXMT_P010129900101_Att_FFFFFF_V1_L1P.FITS")
    !{herspgen_text}
phalist.close()

herspgen : HXMT HE task, herspgen is running
herspgen : HERspGen: Attitude and Alignment correction of detector '0'  is 0.985351 !
herspgen : HXMT HE task, herspgen is running successfully!
herspgen : ##############################################
herspgen : HXMT HE task, herspgen is running
herspgen : HERspGen: Attitude and Alignment correction of detector '1'  is 0.980126 !
herspgen : HXMT HE task, herspgen is running successfully!
herspgen : ##############################################
herspgen : HXMT HE task, herspgen is running
herspgen : HERspGen: Attitude and Alignment correction of detector '2'  is 0.995052 !
herspgen : HXMT HE task, herspgen is running successfully!
herspgen : ##############################################
herspgen : HXMT HE task, herspgen is running
herspgen : HERspGen: Attitude and Alignment correction of detector '3'  is 0.980126 !
herspgen : HXMT HE task, herspgen is running successfully!
herspgen : ##############################################
herspgen

我们产生了17个探测器的响应矩阵。可以查看产生的结果

In [4]:
!ls -tr ./data/HE//he_rsp*

./data/HE//he_rsp_g0.fits  ./data/HE//he_rsp_g9.fits
./data/HE//he_rsp_g1.fits  ./data/HE//he_rsp_g10.fits
./data/HE//he_rsp_g2.fits  ./data/HE//he_rsp_g11.fits
./data/HE//he_rsp_g3.fits  ./data/HE//he_rsp_g12.fits
./data/HE//he_rsp_g4.fits  ./data/HE//he_rsp_g13.fits
./data/HE//he_rsp_g5.fits  ./data/HE//he_rsp_g14.fits
./data/HE//he_rsp_g6.fits  ./data/HE//he_rsp_g15.fits
./data/HE//he_rsp_g7.fits  ./data/HE//he_rsp_g17.fits
./data/HE//he_rsp_g8.fits


### 中能载荷(ME)数据处理

### 低能载荷(LE)数据处理

## 能谱分析